In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [15]:
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/emotions/train'
test_data_dir = '/content/drive/MyDrive/Colab Notebooks/emotions/test'
img_height, img_width = 48, 48  # Adjust these dimensions as needed
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [16]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [18]:
epochs = 15  # You can adjust the number of training epochs
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/15
898/898 [==============================] - 81s 78ms/step - loss: 1.7528 - accuracy: 0.2849 - val_loss: 1.5877 - val_accuracy: 0.3743
Epoch 2/15
898/898 [==============================] - 84s 93ms/step - loss: 1.5742 - accuracy: 0.3848 - val_loss: 1.4201 - val_accuracy: 0.4561
Epoch 3/15
898/898 [==============================] - 70s 78ms/step - loss: 1.4684 - accuracy: 0.4310 - val_loss: 1.3828 - val_accuracy: 0.4716
Epoch 4/15
898/898 [==============================] - 69s 76ms/step - loss: 1.4034 - accuracy: 0.4643 - val_loss: 1.3129 - val_accuracy: 0.4947
Epoch 5/15
898/898 [==============================] - 69s 77ms/step - loss: 1.3629 - accuracy: 0.4790 - val_loss: 1.2686 - val_accuracy: 0.5167
Epoch 6/15
898/898 [==============================] - 69s 76ms/step - loss: 1.3265 - accuracy: 0.4970 - val_loss: 1.2278 - val_accuracy: 0.5272
Epoch 7/15
898/898 [==============================] - 80s 89ms/step - loss: 1.3054 - accuracy: 0.5056 - val_loss: 1.2087 - val_accuracy:

In [19]:
model.save('emotion_recognition_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
# Load the saved model
loaded_model = keras.models.load_model('emotion_recognition_model.h5')

# Load and preprocess the image you want to predict
image_path = '/content/drive/MyDrive/Colab Notebooks/emotions/test/im4.png'  # Replace with the path to your image
img = keras.preprocessing.image.load_img(
    image_path,
    target_size=(img_height, img_width),  # Resize the image to match your model's input size
    color_mode="grayscale"  # Convert the image to grayscale
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Expand dimensions for a single image
img_array /= 255.0  # Normalize pixel values

# Make predictions
predictions = loaded_model.predict(img_array)
emotion_label = list(train_generator.class_indices.keys())[np.argmax(predictions)]

print(f"The predicted emotion is: {emotion_label}")


1/1 [==============================] - 0s 68ms/step
The predicted emotion is: sad


In [34]:
# After training the model, add the testing loop

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

# You can also add more detailed evaluation metrics if needed
# For example, confusion matrix, precision, recall, F1-score, etc.


225/225 [==============================] - 12s 54ms/step - loss: 1.1357 - accuracy: 0.5671
Test loss: 1.1357
Test accuracy: 0.5671
